# Check Course List

Given a list of courses, check if they have been scheduled in Banner. 
This notebook automates the query process and collects results.

The use case for this is the CogSci Graduate Certificate and Minor, which have various courses offered in other departments each semester.

Note that you will need to [install and reference NuGet packages](https://docs.microsoft.com/en-us/nuget/tools/nuget-exe-cli-reference) as appropriate on your system.

## Course List

The list of courses below was generated in a spreadsheet saved as tab delimited. As courses are added/removed from the degree requirements, obviously this should be updated.

In [1]:
let courseList = """degree	department	number
CERT	PSYC	7207
CERT	PSYC	7208
CERT	PSYC	7211
CERT	PSYC	7217
CERT	PSYC	7219
CERT	PSYC	7220
CERT	PSYC	7222
CERT	PSYC	7407
CERT	PSYC	7314
CERT	PSYC	7503
CERT	PSYC	7701
CERT	AUSP	8017
CERT	EECE	7216
CERT	EECE	7262
CERT	EECE	7266
CERT	COMP	6720
CERT	COMP	6730
CERT	COMP	7118
CERT	COMP	7150
CERT	COMP	7282
CERT	COMP	7720
CERT	COMP	7740
CERT	COMP	7745
CERT	COMP	7760
CERT	COMP	7770
CERT	COMP	7780
CERT	COMP	7223
CERT	PSYC	7313
CERT	AUSP	6300
CERT	AUSP	7002
CERT	AUSP	7000
CERT	AUSP	7006
CERT	AUSP	7008
CERT	AUSP	7011
CERT	ENGL	7507
CERT	ENGL	7508
CERT	ENGL	7511
CERT	ENGL	7512
CERT	ENGL	7590
CERT	PSYC	8503
CERT	AUSP	7002
CERT	AUSP	7000
CERT	AUSP	7006
CERT	AUSP	7008
CERT	AUSP	7011
CERT	AUSP	8017
CERT	AUSP	8112
CERT	EECE	7216
CERT	EECE	7262
CERT	EECE	7266
CERT	EECE	7900
CERT	COMP	6720
CERT	COMP	6730
CERT	COMP	7118
CERT	COMP	7150
CERT	COMP	7282
CERT	COMP	7720
CERT	COMP	7740
CERT	COMP	7745
CERT	COMP	7760
CERT	COMP	7770
CERT	COMP	7780
CERT	IDT	8070
CERT	IDT	8074
CERT	IDT	8090
CERT	PHIL	6421
CERT	PSYC	7207
CERT	PSYC	7208
CERT	PSYC	7211
CERT	PSYC	7217
CERT	PSYC	7219
CERT	PSYC	7220
CERT	PSYC	7222
CERT	PSYC	7223
CERT	PSYC	7301
CERT	PSYC	7302
CERT	PSYC	7313
CERT	PSYC	7407
CERT	PSYC	7314
CERT	PSYC	7514
CERT	PSYC	7701
CERT	IIS	6010
CERT	IIS	6011
MINOR	PSYC	1030
MINOR	PSYC	3303
MINOR	PSYC	3530
MINOR	IIS	1010
MINOR	COMP	1900
MINOR	PHIL	3460
MINOR	PHIL	4421
MINOR	PHIL	4661
MINOR	ECON	4512
MINOR	COMP	2700
MINOR	COMP	4720
MINOR	MATH	2120
MINOR	MATH	4083
MINOR	ENGL	3511"""

The below script needs to be able to find the current output cell; this is an easy method to get it.

## Basic Banner Query

The code below describes a basic Banner POST

In [2]:
#r "/z/aolney/repos/FSharp.Data.2.3.2/lib/net40/FSharp.Data.dll"
open FSharp.Data
open System

let QueryClass (dept:string) (number:string) (term:string) =
    let result = 
        Http.RequestString( 
            //"https://banssbprod.memphis.edu/pls/PROD/bwckschd.p_get_crse_unsec",
            "https://ssb.bannerprod.memphis.edu/prod/bwckschd.p_get_crse_unsec",
            query=[
                "term_in", term.Trim();
                "sel_subj","dummy";
                "sel_day","dummy";
                "sel_schd","dummy";
                "sel_insm","dummy";
                "sel_camp","dummy";
                "sel_levl","dummy";
                "sel_sess","dummy";
                "sel_instr","dummy";
                "sel_ptrm","dummy";
                "sel_attr","dummy";
                "sel_subj",dept.Trim();
                "sel_crse",number.Trim();
                "sel_title","";
                "sel_insm","%";
                "sel_from_cred","";
                "sel_to_cred","";
                "sel_camp","%";
                "sel_levl","%";
                "sel_ptrm","%";
                "sel_instr","%";
                "sel_attr","%";
                "begin_hh","0";
                "begin_mi","0";
                "begin_ap","a";
                "end_hh","0";
                "end_mi","0";
                "end_ap","a" ], 
            httpMethod="POST")
        //maybe do some processing here to get object?
    let classFound = result.Contains("No classes were found that meet your search criteria") |> not
    //
    classFound,result

let GetTitle( result : string ) =
    result.Split('\n')
    |> Seq.filter( fun x -> x.Contains("ddtitle") )
    |> Seq.map( fun x ->
        let s = x.Split([|'>';'-'|])
        s.[2].Trim()
        )
    |> Seq.head

## Check Course List

Using the above list and query structure, process the list

In [22]:
//Banner Term, e.g. 201680 for Fall 2016; 201610 for Spring
let term = "202110"

let classes = 
    //"courses-to-check.csv"
    //|> System.IO.File.ReadAllLines
    courseList.Split('\n')
    |> Seq.skip 1 //skip header
    |> Seq.map( fun x -> 
        let s = x.Split('\t')
        let degree = s.[0].Trim()
        let dept = s.[1].Trim()
        let id = s.[2].Trim()
        let classFound, result = QueryClass dept id term
        let title = if classFound then GetTitle( result ) else ""
        (degree,dept,id,title,classFound.ToString())
        //x + "\t" + title + "\t" + classFound.ToString() //example if we wanted to write to file instead
        )
    |> Seq.distinct
    |> Seq.sortBy( fun (degree,dept,id,title,exists) -> exists, degree, dept, id )
    |> Seq.map( fun (degree,dept,id,title,exists) ->  [|degree;dept;id;title;exists|] )
    |> Seq.toArray

//example for file output
//let output = new ResizeArray<string>()
//output.Add("degree\tdepartment\tnumber\tavailable-" + term )
//output.AddRange( classes )
//System.IO.File.WriteAllLines( "checked-courses.csv", output )

//ifsharp f# kernel table output
// {
//     Columns = [|"degree";"department";"number";"title";"available-" + term |]
//     Rows = classes
// }

//dotnet f# kernel table output
Formatter.ListExpansionLimit <- 100
display( classes |> Array.map( fun a -> {|Degree=a.[0];Dept=a.[1];Id=a.[2];Title=a.[3];Exists=a.[4]|}))

index,Degree,Dept,Exists,Id,Title
0,CERT,AUSP,False,6300,
1,CERT,AUSP,False,7002,
2,CERT,AUSP,False,7006,
3,CERT,AUSP,False,7008,
4,CERT,AUSP,False,7011,
5,CERT,AUSP,False,8017,
6,CERT,AUSP,False,8112,
7,CERT,COMP,False,6730,
8,CERT,COMP,False,7150,
9,CERT,COMP,False,7223,
